Goals
- calc sp500 5 yr returns
- calc sp20 5 yr returns 
- store data in mongodb
- create fastapi app that returns calculated data from mongodb
- create visualisation dashboard which gets data from fastapi
- deploy it all on pythonanywhere /docker kubernetes 
- airflow or sth else to do daily runs of the calc apps that will store data in mongodb

In [2]:
import yfinance as yf

In [9]:
import yfinance as yf
import datetime

# Define the start and end dates
start_date = datetime.datetime.now() - datetime.timedelta(days=5*365)
end_date = datetime.datetime.now()

# Get S&P 500 data
sp500 = yf.Ticker('^GSPC')
sp500_data = sp500.history(start=start_date, end=end_date)


In [12]:
sp500_returns = (sp500_data['Close'] / sp500_data['Close'].shift(1)) - 1

Date
2018-09-04 00:00:00-04:00         NaN
2018-09-05 00:00:00-04:00   -0.002803
2018-09-06 00:00:00-04:00   -0.003652
2018-09-07 00:00:00-04:00   -0.002213
2018-09-10 00:00:00-04:00    0.001898
                               ...   
2023-08-28 00:00:00-04:00    0.006265
2023-08-29 00:00:00-04:00    0.014508
2023-08-30 00:00:00-04:00    0.003833
2023-08-31 00:00:00-04:00   -0.001597
2023-09-01 00:00:00-04:00    0.001799
Name: Close, Length: 1258, dtype: float64